In [1]:
import hw2
import requests
import json
import ssl
from IPython.display import display, JSON
import pandas as pd



In [40]:
##special_case_differ3'utr
spliced_data_path1=r"..\raw_data\hw5_data\spliced+UTRTranscriptSequence_B0041.11.2.fasta"
unspliced_data_path1=r"..\raw_data\hw5_data\unspliced+UTRTranscriptSequence_B0041.11.2.fasta"
##special_case_differ5'Utr
spliced_data_path2=r"..\raw_data\hw5_data\spliced+UTRTranscriptSequence_B0041.2a.1.fasta"
unspliced_data_path2=r"..\raw_data\hw5_data\unspliced+UTRTranscriptSequence_B0041.2a.1.fasta"
#normal_case
spliced_data_path3=r"..\raw_data\hw5_data\spliced+UTRTranscriptSequence_Y40B10A.2a.1.fasta"
unspliced_data_path3=r"..\raw_data\hw5_data\unspliced+UTRTranscriptSequence_Y40B10A.2a.1.fasta"


In [8]:
def crawler (transcript_id:str):
    url = f'https://wormbase.org/rest/widget/transcript/{transcript_id}/sequences'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
    try:
        if data['fields']['spliced_sequence_context']['data']['strand']=='+':
            table=data['fields']['spliced_sequence_context']['data']['positive_strand']['features']
        else:
            table=data['fields']['spliced_sequence_context']['data']['negative_strand']['features']
        df=pd.DataFrame(table)
        df['length'] = df['stop'] - df['start'] + 1
        df = df[['type', 'start', 'stop' , 'length']]
    
        last_five_prime_utr_stop = df[df['type'] == 'five_prime_UTR']['stop'].max()
        first_three_prime_utr_start = df[df['type'] == 'three_prime_UTR']['start'].min()
        # 计算CDS的start和stop
        cds_start = last_five_prime_utr_stop + 1
        cds_stop = first_three_prime_utr_start - 1
        # 添加CDS行
        df.loc[len(df)] = ['cds',cds_start, cds_stop, cds_stop - cds_start + 1]
    except:
        print('cantnotfind')
    
    return df

In [49]:
def determine_special(unspliced_path:str,spliced_path:str):
    spliced_chip = hw2.chips(hw2.readfile_generate_raw_sequence(spliced_path))
    unspliced_chip=hw2.chips(hw2.readfile_generate_raw_sequence(unspliced_path))
    if (spliced_chip[0]!=unspliced_chip[0]) | (spliced_chip[-1]!=unspliced_chip[-1]):
        print("it is special case")
        prefix = 'UTRTranscriptSequence_'
        start = unspliced_path.find(prefix) + len(prefix)
        end = unspliced_path.find('.fasta')
        transcript_id = unspliced_path[start:end]
        print(transcript_id)
        df = crawler(transcript_id)
    else:
        print("it is normal case")
        prefix = 'UTRTranscriptSequence_'
        start = unspliced_path.find(prefix) + len(prefix)
        end = unspliced_path.find('.fasta')
        transcript_id = unspliced_path[start:end]
        print(transcript_id)
        #______hw2_____
        index_and_name = hw2.generate_indexAndName(unspliced_chip)
        for i in range(len(index_and_name)):
            del index_and_name[i][4]
        df=pd.DataFrame(index_and_name,columns=['Name','Start','End','Length'])
        #______hw3_____
        df = df[df['Name'].str.contains('Intron') == False].reset_index(drop=True)
        exons_df = df[df['Name'].str.contains('Exon')]
    
        for i in range(2, len(exons_df)+1):#reset exon
            df.loc[i, 'Start'] = df.loc[i-1, 'End'] + 1
            df.loc[i, 'End'] = df.loc[i, 'Start'] + df.loc[i, 'Length']-1
        if (df['Name'].str.contains("3'utr")).any():#if need to reset 3'utr
            indices = df.loc[df['Name'] == "3'utr"].index-1
            valid_indices = indices[indices >= 0]
            df.loc[df['Name'] == "3'utr", 'Start']=df.loc[valid_indices,'End'].iloc[0]-df.loc[df['Name'] == "3'utr","Length"].iloc[0]+1
            df.loc[df['Name'] == "3'utr", 'End']=df.loc[df['Name'] == "3'utr", 'Start'].iloc[0] +df.loc[df['Name'] == "3'utr", 'Length'].iloc[0]-1
        #generate cds
        cds_start=df.loc[df['Name'] == "5'utr", 'End'].iloc[0]+1
        cds_end=df.loc[df['Name'] == "3'utr", 'Start'].iloc[0]-1
        df.loc[len(df)]=["CDS",cds_start,cds_end,(cds_end-cds_start+1)]

    return df

In [50]:
determine_special(unspliced_data_path3,spliced_data_path3)

it is normal case
Y40B10A.2a.1


,Name,Start,End,Length
0,5'utr,1,131,131
1,Exon1,1,191,191
2,Exon2,192,522,331
3,Exon3,523,656,134
4,Exon4,657,868,212
5,3'utr,813,868,56
6,CDS,132,812,681
